# Caravan-ESP: datos auxiliares
***

***Autor:** Jesús Casado Rodríguez*<br>
***Fecha:** 16-09-2023*<br>

**Introducción:**<br>
Para compilar los datos de Caravan son necesarias dos tablas:
* Una tabla con el ID, longitud, latitud, nombre y país de cada una de las estaciones. Las coordenadas deben de estar en el sistema WGS84 (EPSG:4326).
* Una tabla con la serie diaria de caudal específico de cada una de las estaciones.

Además, para publicar el conjunto de datos en Caravan es necesario incluir una capa de polígonos con la cuenca de cada una de las estaciones.

En este _notebook_ se cargan las estaciones seleccionadas para incluir en el conjunto de datos Caravan-ESP, y sus series de caudal diario. Las series de caudal se transforma en caudal específico (caudal dividido por área de la cuenca vertiente, mm/d) y se exportan para su posterior utilización en la generación de Caravan-ESP.

**Por hacer:**<br>
En este _notebook_ se eliminan ciertas estaciones del Anuario de Aforos porque se ha visto que las series no son buenas. Habría que traspasar estas estaciones al _notebook_ inicial en el que se hace la selección de estaciones del Anuario de Aforos.

In [ ]:
import os
os.environ['USE_PYGEOS'] = '0'
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
from datetime import datetime, timedelta
import xarray as xr
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf
from tqdm.notebook import tqdm
import yaml
from pathlib import Path
import pickle

from anuario_aforos import plot_caudal
from funciones import dividir_estaciones, dividir_periodo_estudio

## Configuración

In [ ]:
with open("config.yml", "r", encoding='utf8') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)

# ruta donde se ubican los datos del Anurio de Aforos
path_anuario = Path('../../data/anuario_aforos/')
path_camels = Path(cfg['rutas'].get('caravan', '../../data/CAMELS-ES/'))

# inicio y fin del periodo de estudio
start = cfg['periodo'].get('inicio', None)
end = cfg['periodo'].get('final', None)
start, end = [pd.to_datetime(i) for i in [start, end]]

# tamaño de las muestras de entrenamiento y validación
train_size = cfg.get('train_size', .6)
assert 0 < train_size <= 1., '"train_size" debe de ser un valor entre 0 y 1'
val_size = cfg.get('val_size', .2)
assert 0 < val_size <= 1., '"train_size" debe de ser un valor entre 0 y 1'
if train_size + val_size > 1:
    val_size = 1 - train_size
    print(f'El valor de "val_size" fue truncado a {val_size:.2f}')
seed = cfg.get('seed', 0)

## Datos
### Atributos CARAVAN-ESP

In [ ]:
# cargar tabla de atributos de Caravan
atributos = pd.read_csv(path_camels / 'attributes/attributes_caravan_es.csv', index_col=0)
atributos.index = atributos.index.astype(str)
# atributos.index = [id.split('_')[-1] for id in atributos.index]

print('nº de estaciones:\t{0}\nnº de atributos:\t{1}'.format(*atributos.shape))

### Estaciones 

In [ ]:
# cargar estaciones
estaciones = gpd.read_file(path_anuario / 'GIS/estaciones.shp')
estaciones.set_index('indroea', drop=True, inplace=True)
estaciones = estaciones.loc[atributos.index]
estaciones[['ini_cal', 'fin_cal']] = estaciones[['ini_cal', 'fin_cal']].astype(int)

n_estaciones = estaciones.shape[0]
print('nº de estaciones en la capa de puntos:\t{0}'.format(n_estaciones))

In [ ]:
# !!!!
estaciones.loc['9030', 'fin_cal'] = 2010

In [ ]:
# eliminar estaciones 
# borrar = ['2048', '3233', '3251', '3255', '4207', '4212', '4214',
#           '5140', '7112', '7121', '8027', '8092', '8140', '8148', '9087', '9255']
estaciones = estaciones.loc[~estaciones.index.isin(borrar)]

n_estaciones = estaciones.shape[0]
print('nº de estaciones en la capa de puntos:\t{0}'.format(n_estaciones))

In [ ]:
# # exportar
# estaciones.index.name = 'gauge_id'
# estaciones.to_csv(path_camels / f'stations_camelsesp_{n_estaciones}.csv', float_format='%.4f')

In [ ]:
estaciones_lisflood = gpd.read_file(path_anuario / 'GIS' / 'estaciones_seleccion_EFASv5.shp')
estaciones_lisflood.set_index('indroea', drop=True, inplace=True)
estaciones_lisflood = estaciones_lisflood.loc[estaciones.index]
estaciones_lisflood.to_file(path_anuario / 'GIS' / 'estaciones_seleccion_LISFLOOD.shp', index=True)

**Definir estaciones de entrenamiento y validación**

In [ ]:
# # crear listados de estaciones de entrenamiento, validación y evaluación
# cuencas = ['CANTABRICO', 'GALICIA', 'MINHO', 'DUERO', 'TAJO', 'GUADIANA', 'GUADALQUIVIR', 'SEGURA', 'JUCAR', 'EBRO']
Cuencas = estaciones.cuenca.unique()
ids = estaciones.loc[estaciones.cuenca.isin(Cuencas)].index.to_list()
basins = dividir_estaciones(ids,
                            cal=train_size,
                            val=val_size,
                            path=path_camels,
                            seed=seed)

In [ ]:
# for cuenca in Cuencas:
#     basins_cuenca = dividir_estaciones(estaciones.loc[estaciones.cuenca == cuenca].index.to_list(),
#                                        cal=train_size,
#                                        val=val_size,
#                                        path=Path(f'../3_NeuralHydrology/{cuenca}/'),
#                                        seed=seed)

### Subcuencas

In [ ]:
# cargar polígonos de las cuencas de Caravan
cuencas = gpd.read_file(path_anuario / 'GIS/subcuencas_epsg4326.shp')
cuencas.set_index('gauge_id', drop=True, inplace=True)

# recortar cuencas según la tabla de atributos
cuencas = cuencas.loc[estaciones.index, :]
cuencas.index.name = 'gauge_id'

# generar nuevos campos
cuencas['gauge_name'] = estaciones.lugar
cuencas['gauge_lat'] = estaciones.latwgs84
cuencas['gauge_lon'] = estaciones.longwgs84
cuencas['country'] = 'Spain'
cuencas.area_skm = cuencas.area_skm.astype(int)

# eliminar campos
cuencas.drop(['HydroID'], axis=1, inplace=True)

# reordenar
cuencas = cuencas[['gauge_name', 'gauge_lat', 'gauge_lon', 'country', 'area_skm', 'geometry']]

n_cuencas = cuencas.shape[0]
print('nº de cuencas en la capa de polígonos:\t{0}'.format(n_cuencas))

In [ ]:
# exportar
cuencas.to_file(path_camels / f'shapefiles/catchments_camelsesp_{n_cuencas}.shp', driver='ESRI Shapefile')

In [ ]:
# plot estaciones
proj = ccrs.PlateCarree()
fig, ax = plt.subplots(subplot_kw={'projection': proj})
ax.add_feature(cf.NaturalEarthFeature('physical', 'land', '50m', edgecolor=None, facecolor='lightgray'), zorder=0)
ax.set_extent([-9.5, 3.5, 36, 44.5], crs=proj)
cuencas.plot(ax=ax, facecolor='none', edgecolor='w', linewidth=0.4);
ax.scatter(estaciones.geometry.x, estaciones.geometry.y, c='steelblue', s=5, alpha=1, label='Anuario')
ax.set_title('Estaciones Anuario de Aforos')
ax.axis('off');

# plt.savefig(f'{path_plots}estaciones.jpg', dpi=300, bbox_inches='tight');

### Caudal

In [ ]:
# cargar series de caudal
caudal = pd.read_parquet(path_anuario / 'caudal.parquet', columns=estaciones.index)
caudal = caudal.loc[start:end, estaciones.index]

caudal.shape

**Series de caudal específico**

In [ ]:
# calcular caudal específico
caudal_esp = caudal / estaciones.suprest * 3.6 * 24 # mm/d
caudal_esp.index.name = 'date'

In [ ]:
# exportar
n_series = caudal_esp.shape[1]
# caudal_esp.to_parquet(path_out / f'specific_discharge_camelsesp_{n_series}.parquet')
caudal_esp.to_csv(path_camels / f'specific_discharge_camelsesp_{n_series}.csv', float_format='%.2f')

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))
ax.scatter(estaciones.suprest, caudal_esp.mean(), s=4, alpha=.5)
ax.set(xlabel='area (km²)',
       ylabel='caudal específico (mm/d)');

**Definir periodos de calentamiento, validación y test**

In [ ]:
periodos_ds = {}
for stn in ids:
    if stn in basins['train']:
        cal, val = .6, 0
    elif stn in basins['validation']:
        cal, val = 0, .6
    elif stn in basins['test']:
        cal, val = 0, 0
    else:
        continue
    periodos_ds[stn] = dividir_periodo_estudio(caudal[stn], *estaciones.loc[stn, ['ini_cal', 'fin_cal']], cal, val)
periodos_ds = xr.Dataset(periodos_ds).to_array(dim='id')

# definir periodos de calibración, validación y evaluación para cada estación
# periodos_ds = xr.Dataset({stn: dividir_periodo_estudio(caudal[stn], *estaciones.loc[stn, ['ini_cal', 'fin_cal']]) for stn in caudal.columns}).to_array(dim='id')

# reorganizar el diccionario
periodos_dct = {}
for p in periodos_ds.period.data:
    periodos_dct[p] = {}
    for id in basins[p]: #periodos_ds.id.data:
        periodos_dct[p][id] = {f'{key}_dates': [date] for key, date in periodos_ds.sel(period=p, id=id).to_pandas().to_dict().items()}

# guardar los periodos como pickle
for key, dct in periodos_dct.items():
    with open(path_camels / f'periods_{key}.pkl', 'wb') as f:
        pickle.dump(dct, f)

In [ ]:
# periodo completo de cada estación
periodo_completo = {}
for stn in ids:
    periodo_completo[stn] = {'start_dates': [pd.Timestamp(estaciones.loc[stn, 'ini_cal'], 10, 1)],
                             'end_dates': [pd.Timestamp(estaciones.loc[stn, 'fin_cal'], 9, 30)]}
with open(path_camels / f'periods_complete.pkl', 'wb') as f:
    pickle.dump(periodo_completo, f)

In [ ]:
for id in caudal_esp.columns:
    title = '{0} {1} ({3} km²) - {2}'.format(id, *estaciones.loc[id, ['lugar', 'cuenca', 'suprest']])
    plot_caudal(caudal_esp[id],
                inicios=periodos_ds.sel(id=id, date='start').data,
                finales=periodos_ds.sel(id=id, date='end').data,
                title=title, 
                save=f'hidrogramas/{id:04}.jpg')

In [ ]:
# for period, dct1 in periodos_dct.items():
#     for stn, dct2 in dct1.items():
#         for date, ls in dct2.items():
#             print(period, stn, date, ls[0], sep='\t')